In [1]:
# Data Visualization for "To bee or not to bee"
# IG.2412 & IG.2411

import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import joblib
import warnings
warnings.filterwarnings('ignore')

from PIL import Image
import cv2
from tqdm import tqdm
from skimage import measure, feature, color, morphology

from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import (
    classification_report, confusion_matrix, accuracy_score, silhouette_score
)
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE, Isomap
from sklearn.cluster import KMeans, DBSCAN
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC

from scipy.optimize import minimize
from skimage.transform import rotate

# Configuration des chemins
DATA_DIR = "../../train" 
IMAGES_DIR = os.path.join(DATA_DIR, "images")
MASKS_DIR = os.path.join(DATA_DIR, "masks")
EXCEL_FILE = os.path.join(DATA_DIR, "classif.xlsx")

In [2]:
# Add these missing functions to your feature extraction code

# ==================== MISSING FUNCTION 1: compute_best_inscribed_circle ====================

def compute_best_inscribed_circle(mask):
    """
    II. Computation of the best inscribed circle:
    - Choosing a proper initialization relying on the centroid of the mask
    - Defining a loss function to minimize
    - Minimizing the defined loss using the minimize function of Scipy
    """
    from scipy.ndimage import distance_transform_edt
    
    mask_binary = (mask > 0).astype(int)
    if not np.any(mask_binary):
        return 0.0, (0.0, 0.0)
    
    # Distance transform to find distance to nearest edge
    distance_map = distance_transform_edt(mask_binary)
    
    # Initialize with centroid
    props = measure.regionprops(mask_binary)
    if not props:
        return 0.0, (0.0, 0.0)
    
    centroid = props[0].centroid  # (y, x)
    
    # Loss function: negative radius (to maximize radius)
    def loss_function(coords):
        y, x = coords
        if not (0 <= y < distance_map.shape[0] and 0 <= x < distance_map.shape[1]):
            return 1000  # Penalty for out of bounds
        
        radius = distance_map[int(y), int(x)]
        return -radius  # Negative because we want to maximize
    
    # Minimize using scipy
    result = minimize(
        loss_function,
        x0=[centroid[0], centroid[1]],
        bounds=[(0, mask_binary.shape[0]-1), (0, mask_binary.shape[1]-1)],
        method='L-BFGS-B'
    )
    
    y_opt, x_opt = result.x[0], result.x[1]
    max_radius = -result.fun
    best_center = (x_opt, y_opt)  # Return as (x, y)
    
    return float(max_radius), best_center

# ==================== MISSING FUNCTION 2: rotate_image_filled ====================

def rotate_image_filled(image, angle, center):
    """Rotate image around center with filled background"""
    try:
        # Convert center to skimage format (y, x)
        center_skimage = (center[1], center[0])
        rotated = rotate(image, angle, center=center_skimage, preserve_range=True, cval=0)
        return rotated.astype(image.dtype)
    except Exception as e:
        print(f"Rotation error: {e}")
        return image

# ==================== MISSING FUNCTION 3: compute_best_symmetry_plane ====================

def create_symmetric_image(image, center_x):
    """Create symmetric image around vertical line at center_x"""
    symmetric_image = np.fliplr(image)
    return symmetric_image

def compute_best_symmetry_plane(image, mask, circle_center, circle_radius):
    """
    III. Computation of the best symmetry plane:
    - Choosing a proper initialization among rotations
    - Using the filled and proper rotation function
    - Using function to create symmetric around vertical line
    - Defining a loss function to minimize
    - Minimizing using scipy.minimize
    """
    
    # Create circular mask for region of interest
    Y, X = np.ogrid[:mask.shape[0], :mask.shape[1]]
    cx, cy = circle_center
    circle_mask = (X - cx)**2 + (Y - cy)**2 <= circle_radius**2
    
    # Loss function based on difference between image and its symmetric
    def symmetry_loss_function(angle):
        try:
            # Rotate image
            rotated_image = rotate_image_filled(image, angle[0], circle_center)
            # Create symmetric image
            symmetric_image = create_symmetric_image(rotated_image, cx)
            
            # Calculate difference in circular region
            if np.any(circle_mask):
                diff = np.sum((rotated_image[circle_mask].astype(float) - 
                              symmetric_image[circle_mask].astype(float))**2)
                return diff / np.sum(circle_mask)
            else:
                return 1000
        except Exception as e:
            return 1000
    
    # Try multiple initial angles
    initial_angles = [-30, -15, 0, 15, 30]
    best_result = None
    best_loss = float('inf')
    
    for init_angle in initial_angles:
        try:
            result = minimize(
                symmetry_loss_function,
                x0=[init_angle],
                bounds=[(-45, 45)],
                method='L-BFGS-B'
            )
            
            if result.fun < best_loss:
                best_loss = result.fun
                best_result = result
        except:
            continue
    
    if best_result is not None:
        best_angle = best_result.x[0]
        min_loss = best_result.fun
    else:
        best_angle = 0.0
        min_loss = 1000.0
    
    return float(min_loss), float(best_angle)

# ==================== MISSING FUNCTION 4: extract_shape_features ====================

def extract_shape_features(mask):
    """Extract shape features from mask"""
    mask_binary = (mask > 0).astype(int)
    
    props = measure.regionprops(mask_binary)
    if not props:
        return {
            'area': 0, 'perimeter': 0, 'eccentricity': 0, 'solidity': 0, 
            'extent': 0, 'aspect_ratio': 0, 'horizontal_symmetry': 0
        }
    
    prop = props[0]
    
    # Basic shape features
    area = prop.area
    perimeter = prop.perimeter
    eccentricity = prop.eccentricity
    solidity = prop.solidity
    extent = prop.extent
    
    # Aspect ratio
    major_axis_length = prop.major_axis_length
    minor_axis_length = prop.minor_axis_length
    aspect_ratio = major_axis_length / minor_axis_length if minor_axis_length > 0 else 0
    
    # Horizontal symmetry measure
    height, width = mask_binary.shape
    if width >= 2:
        mid = width // 2
        left_half = mask_binary[:, :mid]
        right_half = mask_binary[:, width-mid:]
        right_half_flipped = np.fliplr(right_half)
        
        min_width = min(left_half.shape[1], right_half_flipped.shape[1])
        if min_width > 0:
            intersection = np.sum(left_half[:, :min_width] & right_half_flipped[:, :min_width])
            union = np.sum(left_half[:, :min_width] | right_half_flipped[:, :min_width])
            horizontal_symmetry = intersection / union if union > 0 else 0
        else:
            horizontal_symmetry = 0
    else:
        horizontal_symmetry = 0
    
    return {
        'area': float(area),
        'perimeter': float(perimeter),
        'eccentricity': float(eccentricity),
        'solidity': float(solidity),
        'extent': float(extent),
        'aspect_ratio': float(aspect_ratio),
        'horizontal_symmetry': float(horizontal_symmetry)
    }

# ==================== MISSING FUNCTION 5: calculate_bug_ratio ====================

def calculate_bug_ratio(mask):
    """Calculate ratio of bug pixels to total pixels"""
    bug_pixels = np.sum(mask > 0)
    total_pixels = mask.size
    ratio = bug_pixels / total_pixels if total_pixels > 0 else 0
    return float(ratio)

# ==================== MISSING FUNCTION 6: extract_color_features ====================

def extract_color_features(image, mask):
    """Extract color features (RGB) from image within mask"""
    mask_bool = mask > 0
    
    if not np.any(mask_bool):
        return {f'{color}_{stat}': 0.0 for color in ['r', 'g', 'b'] 
                for stat in ['min', 'max', 'mean', 'median', 'std']}
    
    # Extract pixels within mask for each channel
    r_channel = image[:, :, 0][mask_bool]
    g_channel = image[:, :, 1][mask_bool]
    b_channel = image[:, :, 2][mask_bool]
    
    def get_channel_stats(channel):
        if len(channel) == 0:
            return {'min': 0, 'max': 0, 'mean': 0, 'median': 0, 'std': 0}
        return {
            'min': float(np.min(channel)),
            'max': float(np.max(channel)),
            'mean': float(np.mean(channel)),
            'median': float(np.median(channel)),
            'std': float(np.std(channel))
        }
    
    # Get stats for each channel
    r_stats = get_channel_stats(r_channel)
    g_stats = get_channel_stats(g_channel)
    b_stats = get_channel_stats(b_channel)
    
    color_features = {}
    for stat in ['min', 'max', 'mean', 'median', 'std']:
        color_features[f'r_{stat}'] = r_stats[stat]
        color_features[f'g_{stat}'] = g_stats[stat]
        color_features[f'b_{stat}'] = b_stats[stat]
    
    return color_features

# ==================== MISSING FUNCTION 7: extract_additional_features ====================

def extract_additional_features(image, mask):
    """Extract additional features (texture and Hu moments)"""
    mask_bool = mask > 0
    
    if not np.any(mask_bool):
        return {
            'texture_entropy': 0.0, 'texture_energy': 0.0,
            'hu_moment1': 0.0, 'hu_moment2': 0.0, 'hu_moment3': 0.0, 'hu_moment4': 0.0
        }
    
    # Texture features using Local Binary Pattern
    try:
        gray = cv2.cvtColor(image, cv2.COLOR_RGB2GRAY)
        radius = min(3, min(image.shape[:2]) // 4)
        n_points = 8 * radius
        
        if radius > 0:
            lbp = feature.local_binary_pattern(gray, n_points, radius, method='uniform')
            lbp_bug = lbp[mask_bool]
            
            if len(lbp_bug) > 0:
                n_bins = int(n_points + 2)
                hist, _ = np.histogram(lbp_bug, bins=n_bins, range=(0, n_bins), density=True)
                hist = hist + 1e-10
                texture_entropy = -np.sum(hist * np.log2(hist))
                texture_energy = np.sum(hist ** 2)
            else:
                texture_entropy = 0.0
                texture_energy = 0.0
        else:
            texture_entropy = 0.0
            texture_energy = 0.0
    except:
        texture_entropy = 0.0
        texture_energy = 0.0
    
    # Hu moments
    try:
        moments = cv2.moments(mask.astype(np.uint8))
        hu_moments = cv2.HuMoments(moments).flatten()
        
        hu_moments_safe = []
        for moment in hu_moments[:4]:
            if abs(moment) < 1e-10:
                hu_moments_safe.append(0.0)
            else:
                hu_moments_safe.append(-np.sign(moment) * np.log10(abs(moment)))
        
        while len(hu_moments_safe) < 4:
            hu_moments_safe.append(0.0)
    except:
        hu_moments_safe = [0.0, 0.0, 0.0, 0.0]
    
    return {
        'texture_entropy': float(texture_entropy),
        'texture_energy': float(texture_energy),
        'hu_moment1': float(hu_moments_safe[0]),
        'hu_moment2': float(hu_moments_safe[1]),
        'hu_moment3': float(hu_moments_safe[2]),
        'hu_moment4': float(hu_moments_safe[3])
    }

# ==================== MISSING FUNCTION 8: extract_all_features ====================

def extract_all_features(image_id):
    """Extract all features for a given image"""
    try:
        # Load and clean image/mask
        image, mask = load_image_and_mask(image_id, visualize=True)
        
        if image is None or mask is None or image.size == 0 or mask.size == 0:
            raise ValueError(f"Problem with image or mask for {image_id}")
        
        # Extract basic features
        shape_features = extract_shape_features(mask)
        bug_ratio = calculate_bug_ratio(mask)
        color_features = extract_color_features(image, mask)
        additional_features = extract_additional_features(image, mask)
        
        # Compute inscribed circle
        max_radius, circle_center = compute_best_inscribed_circle(mask)
        
        # Visualize if valid
        if max_radius > 1:
            visualize_inscribed_circle(image, mask, circle_center, max_radius, image_id)
        
        # Compute symmetry plane
        if max_radius > 5:
            symmetry_score, symmetry_angle = compute_best_symmetry_plane(image, mask, circle_center, max_radius)
            visualize_symmetry_plane(image, mask, circle_center, symmetry_angle, image_id)
        else:
            symmetry_score = 0.0
            symmetry_angle = 0.0
        
        # Combine all features
        features = {
            'image_id': int(image_id),
            'bug_ratio': bug_ratio,
            'max_inscribed_radius': max_radius,
            'symmetry_score': symmetry_score,
            'symmetry_angle': symmetry_angle,
            **shape_features,
            **color_features,
            **additional_features
        }
        
        return features
        
    except Exception as e:
        print(f"Error processing image {image_id}: {e}")
        return create_default_features(image_id)

# ==================== MISSING FUNCTION 9: create_default_features ====================

def create_default_features(image_id):
    """Create default features when extraction fails"""
    return {
        'image_id': int(image_id),
        'bug_ratio': 0.0,
        'max_inscribed_radius': 0.0,
        'symmetry_score': 0.0,
        'symmetry_angle': 0.0,
        'area': 0.0, 'perimeter': 0.0, 'eccentricity': 0.0, 'solidity': 0.0,
        'extent': 0.0, 'aspect_ratio': 0.0, 'horizontal_symmetry': 0.0,
        **{f'{color}_{stat}': 0.0 for color in ['r', 'g', 'b'] 
           for stat in ['min', 'max', 'mean', 'median', 'std']},
        'texture_entropy': 0.0, 'texture_energy': 0.0,
        'hu_moment1': 0.0, 'hu_moment2': 0.0, 'hu_moment3': 0.0, 'hu_moment4': 0.0
    }

# ==================== MISSING FUNCTION 10: normalize_features ====================

def normalize_features(features_df):
    """Normalize features using StandardScaler"""
    from sklearn.preprocessing import StandardScaler
    
    non_numeric_cols = ['image_id', 'bug_type', 'species']
    existing_non_numeric = [col for col in non_numeric_cols if col in features_df.columns]
    numeric_cols = [col for col in features_df.columns if col not in existing_non_numeric]
    
    print(f"Numeric columns detected: {len(numeric_cols)}")
    
    if not numeric_cols:
        raise ValueError("No numeric columns found for normalization.")
    
    numeric_data = features_df[numeric_cols]
    numeric_data = numeric_data.replace([np.inf, -np.inf], np.nan).fillna(0)
    
    scaler = StandardScaler()
    features_df_normalized = features_df.copy()
    features_df_normalized[numeric_cols] = scaler.fit_transform(numeric_data)
    
    return features_df_normalized, scaler

In [ ]:
# Feature Extraction for "To bee or not to bee"
# IG.2412 & IG.2411

def load_classification_data():
    """Load classification data from Excel file"""
    df = pd.read_excel(EXCEL_FILE)
    print(f"Data loaded: {len(df)} insects")
    print(df.head())
    return df

def load_image_and_mask(image_id, visualize=False, output_dir="cleaned_samples"):
    """
    I. Quality control: Load and clean an image and its mask according to instructions:
    - Loading the mask and associated image
    - Transforming the mask into a binary image (label 1 being the insect, label 0 the background)
    - Computing the connected components of the binary mask
    - Restricting the binary mask to its connected component of highest area
    - Restricting both mask and image to the bounding box of the cleaned binary mask
    """
    image_path = os.path.join(IMAGES_DIR, f"{image_id}.jpg")
    mask_path = os.path.join(MASKS_DIR, f"binary_{image_id}.tif")

    # Check file existence
    if not os.path.exists(image_path):
        raise FileNotFoundError(f"Image not found: {image_path}")
    if not os.path.exists(mask_path):
        raise FileNotFoundError(f"Mask not found: {mask_path}")

    # Loading the mask and associated image
    image = np.array(Image.open(image_path).convert("RGB"))
    mask = np.array(Image.open(mask_path).convert("L"))
    
    # Transforming the mask into a binary image
    mask_binary = (mask > 0).astype(np.uint8)

    # Computing the connected components of the binary mask
    labeled_mask = measure.label(mask_binary)
    props = measure.regionprops(labeled_mask)

    if not props:
        raise ValueError(f"No connected components found in mask for image {image_id}")

    # Restricting to largest connected component
    largest_region = max(props, key=lambda x: x.area)
    cleaned_mask = (labeled_mask == largest_region.label).astype(np.uint8)

    # Cropping to bounding box
    minr, minc, maxr, maxc = largest_region.bbox
    cleaned_mask_cropped = cleaned_mask[minr:maxr, minc:maxc]
    cropped_image = image[minr:maxr, minc:maxc]

    # Visualization
    if visualize:
        os.makedirs(output_dir, exist_ok=True)
        
        fig, axes = plt.subplots(2, 2, figsize=(12, 10))
        
        # Original Image
        axes[0,0].imshow(image)
        axes[0,0].set_title(f"Original Image {image_id}")
        axes[0,0].axis('off')
        
        # Original Mask
        axes[0,1].imshow(mask, cmap='gray')
        axes[0,1].set_title("Original Mask")
        axes[0,1].axis('off')
        
        # Cleaned & Cropped Image
        axes[1,0].imshow(cropped_image)
        axes[1,0].set_title("Cleaned & Cropped Image")
        axes[1,0].axis('off')
        
        # Cleaned & Cropped Mask
        axes[1,1].imshow(cleaned_mask_cropped, cmap='gray')
        axes[1,1].set_title("Cleaned & Cropped Mask")
        axes[1,1].axis('off')
        
        plt.tight_layout()
        plt.savefig(os.path.join(output_dir, f"quality_control_{image_id}.png"), dpi=150, bbox_inches='tight')
        plt.close()

    return cropped_image, cleaned_mask_cropped

def visualize_inscribed_circle(image, mask, center, radius, image_id, output_dir="visual_inspection"):
    """Visualize the inscribed circle results"""
    try:
        os.makedirs(output_dir, exist_ok=True)
        
        vis = image.copy()
        cx, cy = int(center[0]), int(center[1])
        
        # Draw inscribed circle in green
        vis = cv2.circle(vis, (cx, cy), int(radius), (0, 255, 0), 2)
        # Draw center in red
        vis = cv2.circle(vis, (cx, cy), 3, (255, 0, 0), -1)
        
        plt.figure(figsize=(8, 6))
        plt.imshow(vis.astype(np.uint8))
        plt.title(f"Best Inscribed Circle - Image {image_id}\nRadius: {radius:.2f}, Center: ({cx}, {cy})")
        plt.axis('off')
        
        vis_path = os.path.join(output_dir, f"inscribed_circle_{image_id}.png")
        plt.savefig(vis_path, bbox_inches='tight', dpi=150)
        plt.close()
        
    except Exception as e:
        print(f"Error visualizing circle for {image_id}: {e}")

def visualize_symmetry_plane(image, mask, center, angle, image_id, output_dir="visual_symmetry"):
    """Visualize symmetry plane results"""
    try:
        os.makedirs(output_dir, exist_ok=True)
        
        rotated_img = rotate_image_filled(image, angle, center)
        vis_rotated = rotated_img.copy()
        cx = int(center[0])
        h, w = vis_rotated.shape[:2]
        
        # Draw vertical axis in cyan
        vis_rotated = cv2.line(vis_rotated, (cx, 0), (cx, h-1), (0, 255, 255), 2)
        
        plt.figure(figsize=(8, 6))
        plt.imshow(vis_rotated.astype(np.uint8))
        plt.title(f"Best Symmetry Plane - Image {image_id}\nOptimal angle: {angle:.2f}°")
        plt.axis('off')
        
        vis_path = os.path.join(output_dir, f"symmetry_plane_{image_id}.png")
        plt.savefig(vis_path, bbox_inches='tight', dpi=150)
        plt.close()
        
    except Exception as e:
        print(f"Error visualizing symmetry for {image_id}: {e}")

def extract_features_for_all_images(df):
    """Extract features for all images"""
    all_features = []
    
    for idx, row in tqdm(df.iterrows(), total=len(df), desc="Extracting features"):
        image_id = row['ID']
        try:
            features = extract_all_features(image_id)
            
            # Add class labels
            features['bug_type'] = row['bug type']
            if 'species' in row:
                features['species'] = row['species']
            
            all_features.append(features)
            
        except Exception as e:
            print(f"Error processing image {image_id}: {e}")
            default_features = create_default_features(image_id)
            default_features['bug_type'] = row['bug type']
            if 'species' in row:
                default_features['species'] = row['species']
            all_features.append(default_features)
    
    return pd.DataFrame(all_features)

if __name__ == "__main__":
    print("=== Starting feature extraction ===")
    
    # Create output directories
    os.makedirs("cleaned_samples", exist_ok=True)
    os.makedirs("visual_inspection", exist_ok=True)
    os.makedirs("visual_symmetry", exist_ok=True)
    
    try:
        # Load data
        classification_df = load_classification_data()
        
        # Feature extraction
        print("Extracting features...")
        features_df = extract_features_for_all_images(classification_df)
        
        if features_df.empty:
            raise ValueError("No features extracted!")
        
        print(f"Features extracted for {len(features_df)} images")
        print(f"Columns: {list(features_df.columns)}")
        
        # Check symmetry angle
        if 'symmetry_angle' in features_df.columns:
            print(f"Symmetry angles - Min: {features_df['symmetry_angle'].min():.2f}°, Max: {features_df['symmetry_angle'].max():.2f}°")
            print(f"Images with non-zero angle: {sum(features_df['symmetry_angle'] != 0)}")
        
        # Save raw features
        features_df.to_csv("../features_raw.csv", index=False)
        print("Raw features saved to 'features_raw.csv'")
        
        # Normalize
        normalized_features_df, scaler = normalize_features(features_df)
        normalized_features_df.to_csv("../features_normalized.csv", index=False)
        joblib.dump(scaler, "scaler.pkl")
        print("Normalized features saved to 'features_normalized.csv'")
        
        # Final statistics
        print(f"\n=== SUMMARY ===")
        print(f"Processed images: {len(normalized_features_df)}")
        print(f"Features per image: {len([col for col in normalized_features_df.columns if col not in ['image_id', 'bug_type', 'species']])}")
        
        if 'bug_type' in normalized_features_df.columns:
            print(f"\nClass distribution:")
            print(normalized_features_df['bug_type'].value_counts())
        
        # Compliance verification
        print(f"\n=== COMPLIANCE CHECK ===")
        
        # I. Quality control
        quality_files = len([f for f in os.listdir("cleaned_samples") if f.startswith("quality_control_")])
        print(f"✓ I. Quality control: {quality_files} visualizations in 'cleaned_samples/'")
        
        # II. Best inscribed circle
        circle_files = len([f for f in os.listdir("visual_inspection") if f.startswith("inscribed_circle_")])
        print(f"✓ II. Best inscribed circle: {circle_files} visualizations in 'visual_inspection/'")
        
        # III. Best symmetry plane
        symmetry_files = len([f for f in os.listdir("visual_symmetry") if f.startswith("symmetry_plane_")])
        print(f"✓ III. Best symmetry plane: {symmetry_files} visualizations in 'visual_symmetry/'")
        
        # IV. Features extraction
        print(f"✓ IV. Features extraction: {len(normalized_features_df)} feature sets")
        
        # Verify symmetry angle calculation
        if 'symmetry_angle' in features_df.columns:
            non_zero_angles = sum(abs(features_df['symmetry_angle']) > 0.1)
            print(f"✓ Symmetry angles calculated: {non_zero_angles}/{len(features_df)} with significant angle")
            
            if non_zero_angles == 0:
                print("⚠️  WARNING: All symmetry angles are 0°. Check implementation.")
        
        print(f"\n=== EXTRACTION COMPLETED SUCCESSFULLY ===")
        print(f"Generated files:")
        print(f"- features_raw.csv: raw features")
        print(f"- features_normalized.csv: normalized features")
        print(f"- scaler.pkl: normalization object")
        print(f"- cleaned_samples/: cleaning visualizations")
        print(f"- visual_inspection/: inscribed circle visualizations")
        print(f"- visual_symmetry/: symmetry plane visualizations")
        
    except Exception as e:
        print(f"General error: {e}")
        import traceback
        traceback.print_exc()

def analyze_extracted_features():
    """IV. Represent the associated values in a 2D space. Comment."""
    try:
        # Load normalized data
        if not os.path.exists("../features_normalized.csv"):
            print("Please run feature extraction first")
            return
        
        df = pd.read_csv("../features_normalized.csv")
        
        # Separate features from labels
        feature_cols = [col for col in df.columns if col not in ['image_id', 'bug_type', 'species']]
        X = df[feature_cols].values
        
        if 'bug_type' in df.columns:
            y = df['bug_type']
        else:
            y = None
        
        print(f"Analyzing {len(feature_cols)} features across {len(df)} images")
        
        # Create 2D visualizations
        fig, axes = plt.subplots(2, 2, figsize=(15, 12))
        
        # 1. PCA
        pca = PCA(n_components=2)
        X_pca = pca.fit_transform(X)
        
        if y is not None:
            scatter = axes[0,0].scatter(X_pca[:, 0], X_pca[:, 1], c=y.astype('category').cat.codes, cmap='viridis', alpha=0.7)
            axes[0,0].set_title(f'PCA (Explained variance: {pca.explained_variance_ratio_.sum():.3f})')
        else:
            axes[0,0].scatter(X_pca[:, 0], X_pca[:, 1], alpha=0.7)
            axes[0,0].set_title('PCA')
        axes[0,0].set_xlabel(f'PC1 ({pca.explained_variance_ratio_[0]:.3f})')
        axes[0,0].set_ylabel(f'PC2 ({pca.explained_variance_ratio_[1]:.3f})')
        
        # 2. t-SNE
        try:
            tsne = TSNE(n_components=2, random_state=42, perplexity=min(30, len(df)//4))
            X_tsne = tsne.fit_transform(X)
            
            if y is not None:
                axes[0,1].scatter(X_tsne[:, 0], X_tsne[:, 1], c=y.astype('category').cat.codes, cmap='viridis', alpha=0.7)
            else:
                axes[0,1].scatter(X_tsne[:, 0], X_tsne[:, 1], alpha=0.7)
            axes[0,1].set_title('t-SNE')
            axes[0,1].set_xlabel('t-SNE 1')
            axes[0,1].set_ylabel('t-SNE 2')
        except Exception as e:
            axes[0,1].text(0.5, 0.5, f't-SNE error: {str(e)[:50]}...', ha='center', va='center')
            axes[0,1].set_title('t-SNE (Error)')
        
        # 3. Specific geometric features
        if 'max_inscribed_radius' in df.columns and 'symmetry_angle' in df.columns:
            if y is not None:
                scatter = axes[1,0].scatter(df['max_inscribed_radius'], df['symmetry_angle'], 
                                         c=y.astype('category').cat.codes, cmap='viridis', alpha=0.7)
            else:
                axes[1,0].scatter(df['max_inscribed_radius'], df['symmetry_angle'], alpha=0.7)
            axes[1,0].set_xlabel('Inscribed circle radius (normalized)')
            axes[1,0].set_ylabel('Symmetry angle (normalized)')
            axes[1,0].set_title('Geometric features')
        
        # 4. Shape features
        if 'aspect_ratio' in df.columns and 'solidity' in df.columns:
            if y is not None:
                scatter = axes[1,1].scatter(df['aspect_ratio'], df['solidity'], 
                                         c=y.astype('category').cat.codes, cmap='viridis', alpha=0.7)
            else:
                axes[1,1].scatter(df['aspect_ratio'], df['solidity'], alpha=0.7)
            axes[1,1].set_xlabel('Aspect ratio (normalized)')
            axes[1,1].set_ylabel('Solidity (normalized)')
            axes[1,1].set_title('Shape features')
        
        # Add legend if we have classes
        if y is not None and len(y.unique()) <= 10:
            handles = [plt.Line2D([0], [0], marker='o', color='w', 
                                markerfacecolor=plt.cm.viridis(i/len(y.unique())), 
                                markersize=8, label=label) 
                      for i, label in enumerate(sorted(y.unique()))]
            fig.legend(handles, sorted(y.unique()), loc='center right', bbox_to_anchor=(1.15, 0.5))
        
        plt.tight_layout()
        plt.savefig("features_2d_analysis.png", dpi=150, bbox_inches='tight')
        plt.close()
        
        print("2D analysis saved to 'features_2d_analysis.png'")
        
        # Statistical analysis
        print(f"\n=== FEATURE ANALYSIS COMMENTS ===")
        
        if y is not None:
            print(f"Number of classes: {len(y.unique())}")
            print(f"Classes: {sorted(y.unique())}")
            
            # Estimate separability
            from sklearn.model_selection import cross_val_score
            from sklearn.linear_model import LogisticRegression
            
            try:
                clf = LogisticRegression(max_iter=1000, random_state=42)
                scores = cross_val_score(clf, X, y, cv=5, scoring='accuracy')
                print(f"Estimated separability (cross-validation accuracy): {scores.mean():.3f} ± {scores.std():.3f}")
                
                if scores.mean() > 0.7:
                    print("✓ Features appear discriminative for separating classes")
                elif scores.mean() > 0.5:
                    print("⚠️  Features show moderate separability")
                else:
                    print("❌ Current features don't appear very discriminative")
                    
            except Exception as e:
                print(f"Could not calculate separability: {e}")
        
        # Feature importance analysis
        print(f"\nFeatures with highest variance:")
        feature_vars = np.var(X, axis=0)
        top_features_idx = np.argsort(feature_vars)[-5:]
        for idx in reversed(top_features_idx):
            print(f"- {feature_cols[idx]}: variance = {feature_vars[idx]:.3f}")
        
        return df
        
    except Exception as e:
        print(f"Analysis error: {e}")
        import traceback
        traceback.print_exc()

=== Starting feature extraction ===
Data loaded: 250 insects
   ID bug type         species
0   1      Bee  Apis mellifera
1   2      Bee  Apis mellifera
2   3      Bee  Apis mellifera
3   4      Bee  Apis mellifera
4   5      Bee  Apis mellifera
Extracting features...


Extracting features:  28%|██▊       | 69/250 [14:03<30:12, 10.01s/it]

Error processing image 69: Unable to allocate 732. MiB for an array with shape (4000, 6000, 4) and data type float64


Extracting features:  28%|██▊       | 70/250 [14:05<23:03,  7.69s/it]

Error processing image 70: Unable to allocate 732. MiB for an array with shape (4000, 6000, 4) and data type float64


...

<Figure size 1200x1000 with 4 Axes>

MemoryError: Unable to allocate 91.6 MiB for an array with shape (4000, 6000, 4) and data type uint8

<Figure size 1200x1000 with 4 Axes>

MemoryError: Unable to allocate 91.6 MiB for an array with shape (4000, 6000, 4) and data type uint8

<Figure size 1200x1000 with 4 Axes>